# $\alpha,\!\beta$-CROWN Verifier Quick Start Tutorial
$\alpha,\!\beta$-CROWN (alpha-beta-CROWN) is a neural network verifier based on an efficient bound propagation algorithm (CROWN) and branch and bound. It can be accelerated efficiently on GPUs and can scale to relatively large convolutional networks. It also supports a wide range of neural network architectures (e.g., CNN, ResNet, and various activation functions), thanks to the versatile auto_LiRPA library developed by us. α,β-CROWN can provide provable robustness guarantees against adversarial attacks and can also verify other general properties of neural networks.

α,β-CROWN is the winning verifier in VNN-COMP 2021 (International Verification of Neural Networks Competition) with the highest total score, outperforming 11 other neural network verifiers on a wide range of benchmarks. Code can be downloaded at [α,β-CROWN repo](https://github.com/huanzhang12/alpha-beta-CROWN) with detailed and friendly [instructions for usages](https://github.com/huanzhang12/alpha-beta-CROWN/tree/main/docs).

## Installation & Imports
We first install α,β-CROWN with given environment.yml. Note that our library is tested on Pytorch 1.8.2 LTS, and other versions might be incompatible.

Install miniconda

In [ ]:
%%bash
%env PYTHONPATH=
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
%%bash
conda install --channel defaults conda python=3.7 --yes
conda update --channel defaults --all --yes

In [ ]:
import sys
sys.path
!ls /usr/local/lib/python3.7/dist-packages
_ = (sys.path
        .append("/usr/local/lib/python3.7/site-packages"))

clone the alpha-beta-crown repo

In [ ]:
# Uninstall existing Pytorch on Colab, which might be incompatible or buggy.
# !pip uninstall --yes torch torchvision torchaudio torchtext
!git clone https://github.com/huanzhang12/alpha-beta-CROWN.git

Create conda environment according to provided environment.yml

In [ ]:
%%bash
# Remove the old environment, if necessary.
conda env remove --name alpha-beta-crown
conda env create -f alpha-beta-CROWN/complete_verifier/environment.yml  # install all dependents into the alpha-beta-crown environment
# conda activate alpha-beta-crown  # activate the environment

Go to the running folder

In [ ]:
%cd alpha-beta-CROWN/complete_verifier/

/content/alpha-beta-CROWN/complete_verifier


## Tutorial example1: Running MNIST example
Here we define our own config file. Forr instance, we define a testing example for mnist_cnn_a_adv model with epsilon 0.3

In [ ]:
%%writefile -a exp_configs/tutorial_mnist_example.yaml
general:
  mode: verified-acc
model:
  name: mnist_cnn_4layer
  path: models/sdp/mnist_cnn_a_adv.model
data:
  dataset: MNIST
  std: [1.]
  mean: [0.]
specification:
  epsilon: 0.3
attack:
  pgd_restarts: 50
solver:
  beta-crown:
    batch_size: 1024
    iteration: 20
bab:
  timeout: 180

Writing exp_configs/tutorial_mnist_example.yaml


activate the conda environment and run the verification on mnist sample index 21

In [ ]:
%%bash
source activate alpha-beta-crown
# python robustness_verifier.py --config exp_configs/tutorial_mnist_example.yaml --start 0 --end 1
# python robustness_verifier.py --config exp_configs/tutorial_mnist_example.yaml --start 1 --end 2
python robustness_verifier.py --config exp_configs/tutorial_mnist_example.yaml --start 21 --end 22
conda deactivate

Configurations:

general:
  device: cuda
  seed: 100
  conv_mode: patches
  deterministic: false
  double_fp: false
  loss_reduction_func: sum
  record_bounds: false
  mode: verified-acc
  complete_verifier: bab
  enable_incomplete_verification: true
  get_crown_verified_acc: false
model:
  path: models/sdp/mnist_cnn_a_adv.model
  name: mnist_cnn_4layer
data:
  start: 21
  end: 22
  num_outputs: 10
  mean: [0.0]
  std: [1.0]
  pkl_path: null
  dataset: MNIST
  data_filter_path: null
  data_idx_file: null
specification:
  type: lp
  norm: .inf
  epsilon: 0.3
solver:
  alpha-crown:
    lr_alpha: 0.1
    iteration: 100
    share_slopes: false
    no_joint_opt: false
  beta-crown:
    batch_size: 1024
    lr_alpha: 0.01
    lr_beta: 0.05
    lr_decay: 0.98
    optimizer: adam
    iteration: 20
    beta: true
    beta_warmup: true
  mip:
    parallel_solvers: null
    solver_threads: 1
    refine_neuron_timeout: 15
    refine_neuron_time_percentage: 0.8
    early_stop: true
bab:
  max_domai

9913344it [00:00, 136688599.28it/s]        
29696it [00:00, 99484066.76it/s]         
1649664it [00:00, 68799764.48it/s]         
5120it [00:00, 34918433.30it/s]         
/usr/local/envs/alpha-beta-crown/lib/python3.7/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1627336316785/work/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/usr/local/envs/alpha-beta-crown/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our

### More examples

We provide many examples of `α,β-CROWN` in our repository with detailed config files at [here](https://github.com/huanzhang12/alpha-beta-CROWN/tree/main/complete_verifier/exp_configs).

## Customization on your own model and data
One can easily customize the usage of α,β-CROWN to other dataset and self-defined models

### Tutorial example 2: Customization for official CIFAR dataset and self-defined models

In [ ]:
%%writefile your_model_data.py
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import arguments
import numpy as np


def simple_conv_model(in_channel, out_dim):
    """Simple Convolutional model."""
    model = nn.Sequential(
        nn.Conv2d(in_channel, 16, 4, stride=2, padding=0),
        nn.ReLU(),
        nn.Conv2d(16, 32, 4, stride=2, padding=0),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(32*6*6,100),
        nn.ReLU(),
        nn.Linear(100, out_dim)
    )
    return model


def cifar10(eps, use_bounds=False):
    """Example dataloader. For MNIST and CIFAR you can actually use existing ones in utils.py."""
    assert eps is not None
    database_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), 'datasets')
    # You can access the mean and std stored in config file.
    mean = torch.tensor(arguments.Config["data"]["mean"])
    std = torch.tensor(arguments.Config["data"]["std"])
    normalize = transforms.Normalize(mean=mean, std=std)
    test_data = datasets.CIFAR10(database_path, train=False, download=True, transform=transforms.Compose([transforms.ToTensor(), normalize]))
    # Load entire dataset.
    testloader = torch.utils.data.DataLoader(test_data, batch_size=10000, shuffle=False, num_workers=4)
    X, labels = next(iter(testloader))
    if use_bounds:
        # Option 1: for each example, we return its element-wise lower and upper bounds.
        # If you use this option, set --spec_type ("specifications"->"type" in config) to 'bound'.
        absolute_max = torch.reshape((1. - mean) / std, (1, -1, 1, 1))
        absolute_min = torch.reshape((0. - mean) / std, (1, -1, 1, 1))
        # Be careful with normalization.
        new_eps = torch.reshape(eps / std, (1, -1, 1, 1))
        data_max = torch.min(X + new_eps, absolute_max)
        data_min = torch.max(X - new_eps, absolute_min)
        # In this case, the epsilon does not matter here.
        ret_eps = None
    else:
        # Option 2: return a single epsilon for all data examples, as well as clipping lower and upper bounds.
        # Set data_max and data_min to be None if no clip. For CIFAR-10 we clip to [0,1].
        data_max = torch.reshape((1. - mean) / std, (1, -1, 1, 1))
        data_min = torch.reshape((0. - mean) / std, (1, -1, 1, 1))
        if eps is None:
            raise ValueError('You must specify an epsilon')
        # Rescale epsilon.
        ret_eps = torch.reshape(eps / std, (1, -1, 1, 1))
    return X, labels, data_max, data_min, ret_eps



Writing your_model_data.py


Use the config file with customized model "simple_conv" and dataset "cifar10"

In [ ]:
%%writefile exp_configs/tutorial_cifar_example.yaml
general:
  mode: verified-acc
model:
  # Use the simple_conv_model() model in "your_model_data.py".
  name: Customized("your_model_data", "simple_conv_model", in_channel=3, out_dim=10)
  path: models/eran/cifar_conv_small_pgd.pth
data:
  # Use the cifar10() loader in "your_model_data.py".
  dataset: Customized("your_model_data", "cifar10")
  mean: [0.4914, 0.4822, 0.4465]
  std: [0.2023, 0.1994, 0.201]
specification:
  epsilon: 0.00784313725  # 2./255.
attack:
  pgd_restarts: 100
solver:
  beta-crown:
    batch_size: 2048
    iteration: 20
bab:
  max_domains: 5000000
  timeout: 300

Writing exp_configs/tutorial_cifar_example.yaml


Then we can just run the verification:

In [ ]:
%%bash
source activate alpha-beta-crown
python robustness_verifier.py --config exp_configs/tutorial_cifar_example.yaml --start 3 --end 4
conda deactivate

Configurations:

general:
  device: cuda
  seed: 100
  conv_mode: patches
  deterministic: false
  double_fp: false
  loss_reduction_func: sum
  record_bounds: false
  mode: verified-acc
  complete_verifier: bab
  enable_incomplete_verification: true
  get_crown_verified_acc: false
model:
  path: models/eran/cifar_conv_small_pgd.pth
  name: 'Customized("your_model_data", "simple_conv_model", in_channel=3, out_dim=10)'
data:
  start: 3
  end: 4
  num_outputs: 10
  mean: [0.4914, 0.4822, 0.4465]
  std: [0.2023, 0.1994, 0.201]
  pkl_path: null
  dataset: 'Customized("your_model_data", "cifar10")'
  data_filter_path: null
  data_idx_file: null
specification:
  type: lp
  norm: .inf
  epsilon: 0.00784313725
solver:
  alpha-crown:
    lr_alpha: 0.1
    iteration: 100
    share_slopes: false
    no_joint_opt: false
  beta-crown:
    batch_size: 2048
    lr_alpha: 0.01
    lr_beta: 0.05
    lr_decay: 0.98
    optimizer: adam
    iteration: 20
    beta: true
    beta_warmup: true
  mip:
    par

170499072it [00:05, 29212209.59it/s]                               
/usr/local/envs/alpha-beta-crown/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### Tutorial example3: Customization for arbitrary data verification
Besides official data loader, one can also easily use α,β-CROWN to verify arbitrarily defined dataset and model. You can even define element-wise perturbation ranges like here.

In [ ]:
%%writefile -a your_model_data.py
def two_relu_toy_model(in_dim=2, out_dim=2):
    """A very simple model, 2 inputs, 2 ReLUs, 2 outputs"""
    model = nn.Sequential(
        nn.Linear(in_dim, 2),
        nn.ReLU(),
        nn.Linear(2, out_dim)
    )
    """[relu(x+2y)-relu(2x+y)+2, 0*relu(2x-y)+0*relu(-x+y)]"""
    model[0].weight.data = torch.tensor([[1., 2.], [2., 1.]])
    model[0].bias.data = torch.tensor([0., 0.])
    model[2].weight.data = torch.tensor([[1., -1.], [0., 0.]])
    model[2].bias.data = torch.tensor([2., 0.])
    return model

def simple_box_data():
    """a customized box data: x=[-1.5, 1], y=[-1, 1.5]"""
    X = torch.tensor([[0., 0.]]).float()
    labels = torch.tensor([0]).long()
    # customized element-wise upper bounds
    data_max = torch.tensor([[1., 1.5]]).reshape(1, -1)
    # customized element-wise lower bounds
    data_min = torch.tensor([[-1.5, -1.]]).reshape(1, -1)
    eps = None
    return X, labels, data_max, data_min, eps

Appending to your_model_data.py


Use the config file with customized model "simple_box" and dataset "simple_box_data"

In [ ]:
%%writefile exp_configs/tutorial_simple_box_example.yaml
general:
  mode: verified-acc
model:
  # Use the two_relu_toy_model() model in "your_model_data.py".
  name: Customized("your_model_data", "two_relu_toy_model", in_dim=2, out_dim=2)
data:
  # Use the simple_box_data() loader in "your_model_data.py".
  dataset: Customized("your_model_data", "simple_box_data")
  num_outputs: 2
specification:
  type: bound
attack:
  pgd_order: skip
solver:
  beta-crown:
    batch_size: 2048
    iteration: 20
bab:
  timeout: 300
  branching:
    method: fsb

Writing exp_configs/tutorial_simple_box_example.yaml


Then we can just run the verification:

In [ ]:
%%bash
source activate alpha-beta-crown
python robustness_verifier.py --config exp_configs/tutorial_simple_box_example.yaml
conda deactivate

Configurations:

general:
  device: cuda
  seed: 100
  conv_mode: patches
  deterministic: false
  double_fp: false
  loss_reduction_func: sum
  record_bounds: false
  mode: verified-acc
  complete_verifier: bab
  enable_incomplete_verification: true
  get_crown_verified_acc: false
model:
  path: null
  name: 'Customized("your_model_data", "two_relu_toy_model", in_dim=2, out_dim=2)'
data:
  start: 0
  end: 10000
  num_outputs: 2
  mean: 0.0
  std: 1.0
  pkl_path: null
  dataset: 'Customized("your_model_data", "simple_box_data")'
  data_filter_path: null
  data_idx_file: null
specification:
  type: bound
  norm: .inf
  epsilon: null
solver:
  alpha-crown:
    lr_alpha: 0.1
    iteration: 100
    share_slopes: false
    no_joint_opt: false
  beta-crown:
    batch_size: 2048
    lr_alpha: 0.01
    lr_beta: 0.05
    lr_decay: 0.98
    optimizer: adam
    iteration: 20
    beta: true
    beta_warmup: true
  mip:
    parallel_solvers: null
    solver_threads: 1
    refine_neuron_timeout: 15
